In [9]:
import requests
from personal_info import access_token
import json

In [3]:

# r = requests.get('https://api.github.com/versions')
# r.status_code
# r.json()
# r.headers['Content-Type']

In [4]:
# especificando a versão da API
headers = {'X-GitHub-Api-Version': '2022-11-28',
           'Authorization': 'Bearer' + access_token}
api_base_url = 'https://api.github.com'
owner = 'amzn' # username from whom we will extract the data
url = f"{api_base_url}/users/{owner}/repos"

url

'https://api.github.com/users/amzn/repos'

In [5]:
response = requests.get(url=url, headers=headers)
response.status_code


200

In [6]:
print(type(response.json()))
print(len(response.json()))


<class 'list'>
30


## Paginando os repositórios


In [7]:
url

'https://api.github.com/users/amzn/repos'

In [ ]:
repos_list = []
page_num = 1

while True:
    try:
        url_page = f'{url}?page={page_num}'
        response = requests.get(url_page, headers=headers)
        if len(response.json()) == 0:
            break
        repos_list.append(response.json())
        page_num += 1
    except:
        repos_list.append(None)
        break

In [9]:
# Cada item da lista é uma página, com seus respectivos repos
print(len(repos_list))
print(len(repos_list[0]))



6
30


## Transformando os dados

### Nome dos repos

In [10]:
repos_list[0][2]['name']

'ads-pao-amznjs-gtm-template'

In [11]:
repos_name = []

for page in repos_list:
    for repo in page:
        repos_name.append(repo['name'])

In [12]:
repos_name[:10]

['.github',
 'ads-advanced-tools-docs',
 'ads-pao-amznjs-gtm-template',
 'alexa-coho',
 'alexa-skills-kit-js',
 'amazon-ads-advertiser-audience-normalization-sdk-py',
 'amazon-advertising-api-php-sdk',
 'amazon-codeguru-profiler-for-spark',
 'amazon-frustration-free-setup-certification-tool',
 'amazon-hub-counter-api-docs']

In [13]:
len(repos_name)

158

### Linguagens dos repos

In [14]:
repos_list[0][29]['language']

'Python'

In [15]:
repos_language = []

for page in repos_list:
    for repo in page:
        repos_language.append(repo['language'])

In [16]:
print(repos_language)
print(len(repos_language))

[None, 'Jupyter Notebook', 'Smarty', 'JavaScript', None, 'Python', 'PHP', 'Java', 'Python', 'CSS', 'Java', 'Java', 'Java', 'C#', 'PHP', 'Ruby', 'JavaScript', 'Python', 'PHP', 'Python', 'Jupyter Notebook', 'C#', 'Java', 'JavaScript', 'PHP', 'Ruby', 'C#', 'Java', 'PHP', 'Python', 'Ruby', 'PHP', 'Kotlin', 'PHP', 'Python', 'C', None, 'Swift', 'Python', 'C++', 'Python', 'Go', 'C', 'Python', 'Python', 'Jupyter Notebook', 'Python', 'Python', None, 'Java', 'Kotlin', 'Python', 'Python', 'JavaScript', 'TypeScript', 'Python', 'TypeScript', 'JavaScript', 'TypeScript', 'Python', None, 'Jupyter Notebook', 'Python', 'Python', 'Python', 'Java', 'Jupyter Notebook', 'Python', 'Python', 'Java', 'Objective-C', 'JavaScript', 'TypeScript', 'Java', None, 'Python', 'Python', 'Python', 'Java', 'Java', 'Kotlin', 'Java', 'C#', 'C#', 'JavaScript', 'JavaScript', 'Go', 'Java', 'TypeScript', 'Python', 'C++', None, 'Python', 'Python', 'Java', 'C#', 'HTML', 'Kotlin', None, 'PHP', 'PHP', None, 'JavaScript', 'Python', '

### DataFrame

In [17]:
import pandas as pd

In [18]:
dados_amz = pd.DataFrame()
dados_amz['repository_name'] = repos_name
dados_amz['language'] = repos_language


In [20]:
dados_amz

,repository_name,language
0,.github,None
1,ads-advanced-tools-docs,Jupyter Notebook
2,ads-pao-amznjs-gtm-template,Smarty
3,alexa-coho,JavaScript
4,alexa-skills-kit-js,None
...,...,...
153,zeek-plugin-enip,Zeek
154,zeek-plugin-profinet,Zeek
155,zeek-plugin-s7comm,Zeek
156,zeek-plugin-tds,Zeek


## Salvando o DataFrame

In [21]:
dados_amz.to_csv('amazon.csv')

## Armazenando os dados

### Criando repo com POST

In [11]:
api_base_url = 'https://api.github.com'
url = f'{api_base_url}/user/repos'
# Will identify your account based on the previously used access_token
url

'https://api.github.com/user/repos'

In [ ]:
# Criando as infos do repo
headers = {'X-GitHub-Api-Version': '2022-11-28',
           "Authorization": f"token {access_token}",
           "Accept": "application/vnd.github.v3+json"}


data = {
    'name': 'linguagens-utilizadas',
    'description': 'Repo com linguagem de programação utilizadas pela Amazon.',
    'private': False
}

response = requests.post(url=url, data=json.dumps(data), headers=headers)
response

<Response [201]>

### Formato do Arquivo

In [13]:
import base64

In [ ]:
with open('amazon.csv', 'rb') as file:
    file_content = file.read()

encoded_content = base64.b64encode(file_content)

### Upload do arquivo no GitHub

In [18]:
api_base_url = 'https://api.github.com'
username = 'fFukurou'
repo = 'linguagens-utilizadas'
path = 'amazon.csv'

url = f"{api_base_url}/repos/{username}/{repo}/contents/{path}"
url

'https://api.github.com/repos/fFukurou/linguagens-utilizadas/contents/amazon.csv'

In [19]:
data = {
    'message': 'added amazon.csv',
    'content': encoded_content.decode('utf-8'),
}

response = requests.put(url, json=data, headers=headers)
response.status_code

201